In [ ]:
print ("""TFM: RISK MANAGEMENT IN IBERIAN SPOT MARKET.""")
    

In [ ]:
'''
1. IMPORTING USEFUL PACKAGES.
'''

import pandas as pd
import requests # When you ping a website or portal for information this is called making a request.
import json # JavaScript Object Notation. Storing and Exanging files. 
import os # You can find important information about your location or about the process. 
import win32com.client as win32
%pylab inline 
# Import numpy as np and import matploltlib(plt as pyplot)
import esios # Imports our esios package.
os.getcwd()  #Current working directory.

In [ ]:

#os.chdir('C:\\Users\\user\\Desktop\\ione py') # Working directory. esios.py must be there.
os.chdir('C:\\Users\\ilopez18\\Desktop\\py\\')

In [ ]:
'''
2. BRIEF DESCRIPTION ON API REQUESTS AND SPOT MARKET AGENTS.
'''

esios.header() #  Header API request.

# In esios.py package introduce your own TOKEN.- consultasios@ree.es

In [ ]:
esios.archives_json_dict() #dic with json files.

In [ ]:
esios_indicators = esios.indicators_dict()
esios_indicators

In [ ]:
len(esios_indicators)# Number or indicators.

In [ ]:
'''
lambda operator can have any number of arguments, but it can have only one expression. It cannot contain any statements and it returns a function object which can be assigned to any variable.
'''
#Sort esios_indicators dictionary by key, giving a list. Introducing a 'x[0]' will sort by value, alphabetically.
sorted_esios_indicators = sorted(esios_indicators.items(), key=lambda x: x[1])
sorted_esios_indicators

In [ ]:
programunits = esios.PUs() #Dataframe wich contains all agents present in Spot market.
programunits

In [ ]:
programunits.describe() #short description of Spot Market agents.

In [ ]:
programunits.info()

In [ ]:
#Alternative way to download "UNIDADES DE PROGRAMACION" dataset.
#path ="C:............/Data.csv" #Download "UNIDADES DE PORGRAMACION" from www.esios.ree.es, rename as Data.
#programunits = pd.read_csv(path, sep=';')
#programunits.head()

In [ ]:
#Columns tittles.
programunits.columns.values.tolist()


In [ ]:
#Erase spaces from dataframe head.
programunits.columns = programunits.columns.str.replace(' ','')
programunits

In [ ]:
#Is there any NAN?
programunits.isnull().values.any()

In [ ]:
#Where are this NaN.
programunits.isnull().sum()

In [ ]:
#DISPLAY dataframe cointaining NAN.
programunitsNAN = programunits[programunits.isnull().any(axis=1)]
programunitsNAN

In [ ]:
#change NAN for Porfolio
programunits.fillna('PORFOLIO', inplace=True)
programunits

In [ ]:
#Unique values in column 'ProductionType'
programunits.ProductionType.unique()


In [ ]:
ini, fin = '20180801', '20181025'
lista_indicadores = [541,10249, 460,600,39,74,82, 10027,10034,84,85,10010, 762]

datos = esios.Batch(ini, fin, lista_indicadores)

datos
#X = datos[['541','10249','10027','39','82','74','460','10010','84','85','10034']]
#X = X.fillna(0)


In [ ]:
import esios

In [ ]:
'''

3. REGRESSIONS 

'''
#RANDOM FOREST

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score


ini, fin = '20180801', '20181025'
lista_indicadores = [541,10249, 460,600,39,74,82, 10027,10034,84,85,10010, 762]

datos = esios.Batch(ini, fin, lista_indicadores)


X = datos[['541','10249','10027','39','82','74','460','10010','84','85','10034']]
X = X.fillna(0)
X['hour'] = [x.hour for x in X.index]
X['weekday'] = [x.weekday() for x in X.index]
X['weekend'] = [1 if x>=5 else 0 for x in X['weekday']]
X['dif_demp48'] = X['10027']-X['460']
X['dif_eol48'] = X['10010']-X['541']
X['windpower/Load'] = X['541'] / X['460'] 
X['P48SOLAR']= X['84']+X['85']
X['dif_SOLP48'] = X['P48SOLAR']-X['10034']
X['Residual_Load/Load'] = X['10249'] / X['460'] 

y = datos[['762']]#762

df_corr = pd.merge(X, y, left_index=True, right_index=True)
correl = df_corr.corr()
correl[['762']].sort_values(by='762', ascending=False)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25)

reg = RandomForestRegressor(n_estimators= 5000)#5000 trees
reg.fit(X_train, y_train.values.ravel())
predTest = reg.predict(X_test)

r2_test = r2_score(y_test, predTest)
r2_test

In [ ]:
indicator = [541,10249, 460,39,74,82,10027,10034,84,85,10010, 762]
indicator

In [ ]:
plt.plot( X['541'], marker='.', color='black', linewidth=3, alpha=0.5)
plt.xlabel("'windpower'")
plt.ylabel("MW")

In [ ]:
plt.plot( X['windpower/Load'], marker='.', color='blue', linewidth=3, alpha=0.5)
plt.xlabel("'windpower/Load'")
plt.ylabel("MW/Load")

In [ ]:
plt.margins(xmargin=0.15)
plt.plot( X['P48SOLAR'], marker='.', color='orange', linewidth=3, alpha=0.5)
plt.xlabel("Solar p48 prgram")
plt.ylabel("MW")

In [ ]:
plt.plot( datos[['762']], marker='.', color='green', linewidth=3, alpha=0.5)
plt.xlabel("Balancegrid")
plt.ylabel("MW")

In [ ]:
a = dict(zip(X.columns, reg.feature_importances_)) #input significance.
a

In [ ]:
#imput correlation heatmap. 

import seaborn as sns
%pylab inline

sns.heatmap(datos.corr(),annot = True)

In [ ]:
datos

In [ ]:
#GRADIENT BOOSTING

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score, mean_squared_error

df_corr = pd.merge(X, y, left_index=True, right_index=True)
correl = df_corr.corr()
correl[['762']].sort_values(by='762', ascending=False)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25)

pin = {'learning_rate': 0.1, 'max_depth': 5, 'subsample': 0.8}
gbr = GradientBoostingRegressor(n_estimators=5000, **pin)

gbr.fit(X_train, y_train.values.ravel())                       
predTest = gbr.predict(X_test)
                       
r2_test = r2_score(y_test, predTest)
r2_test


In [ ]:
#K-NEIGHBORS

from sklearn.neighbors import KNeighborsRegressor        

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25)

kn=3
knn = KNeighborsRegressor(n_neighbors = kn)
knn.fit(X_train, y_train.values.ravel())
testPred = knn.predict(X_test)
r2_test = r2_score(y_test, testPred)
knnPred = knn.predict(X_test)

r2_test = r2_score(y_test, knnPred)
r2_test
        
